In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb

from scipy.optimize import differential_evolution


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Load Data

file_name = "../data/train_preprocessed2.csv"
train_df = pd.read_csv(file_name, low_memory = False)

train_df.head()

,A..papers,A.papers,B.papers,C.papers,Dif.countries,Perc_non_australian,Number.people,PHD,Max.years.univ,Grants.succ,...,SEO.11,SEO.12,SEO.13,SEO.14,SEO.15,SEO.16,SEO.17,SEO.18,SEO.19,Grant.Status
0,4.0,2.0,0.0,0.0,1,0.00,1,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1,6.0,12.0,2.0,2.0,1,1.00,1,1.0,20.0,0.0,...,0,0,0,0,0,0,0,0,0,1
2,7.0,20.0,20.0,7.0,2,0.75,4,2.0,50.0,0.0,...,0,0,2,0,0,0,0,0,0,1
3,0.0,3.0,13.0,3.0,1,1.00,2,2.0,15.0,0.0,...,0,0,2,0,0,0,0,0,0,1
4,3.0,0.0,1.0,0.0,1,0.00,1,1.0,10.0,0.0,...,0,0,0,0,0,0,1,0,0,0


In [3]:
#Setup data : Divide Test and Train set

array = train_df.values

data = array[:, 0:70]
target = array[:, 70]

data, target

seed = 7
test_size = 0.2

data_train, data_test, target_train, target_test = train_test_split(data, target, test_size = test_size, random_state = seed)


In [5]:
# Making a XGB Train Model Function

def XGB_Train_Model_using_KFold(x) : 
    min_child_weight = x[0]
    max_depth = x[1]
    gamma = x[2]
    subsample = x[3] 
    colsample_bytree = x[4]
    xgb_params = {
        'n_trees' : 250,
        'eta' : 0.2,
        'max_depth' : int(max_depth),
        'subsample' : max(min(subsample, 1), 0),
        'objective' : 'reg:linear', 
        'base_score' : np.mean(target),
        'eval_metric' : 'auc',
        'silent' : 1,
        'min_child_weight' : int(min_child_weight),
        'gamma' : max(gamma, 0), 
        'colsample_bytree' : max(min(colsample_bytree, 1), 0)
    }
    
    model = xgb.XGBClassifier(**xgb_params)
    
    #n_splits = int(max(n_splits_param, 2))
    kfold = KFold(n_splits = 5, random_state = 7, shuffle = True)
    results = cross_val_score(model, data, target, cv = kfold)
    auc = results.mean()*100
    print("min_child_weight : %.2f%% || max_depth : %.2f%% || gamma : %.2f%% || subsample : %.2f%% || colsample_bytree : %.2f%%" %(x[0], x[1], x[2], x[3], x[4]))
    print("AUC : %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    
    
    return 100 - auc

In [31]:
bounds = [(1,20), (2, 10), (0, 10), (0.5, 1), (0.1, 1)]

result = differential_evolution(XGB_Train_Model_using_KFold, bounds)
result.x, result.fun

min_child_weight : 17.10% || max_depth : 5.93% || gamma : 9.98% || subsample : 0.79% || colsample_bytree : 0.42%
AUC : 86.25% (0.90%)
min_child_weight : 18.42% || max_depth : 7.74% || gamma : 4.28% || subsample : 0.71% || colsample_bytree : 0.32%
AUC : 87.64% (0.81%)
min_child_weight : 1.40% || max_depth : 8.39% || gamma : 8.08% || subsample : 0.80% || colsample_bytree : 0.15%
AUC : 86.70% (0.74%)
min_child_weight : 2.99% || max_depth : 8.25% || gamma : 3.34% || subsample : 0.88% || colsample_bytree : 0.13%
AUC : 87.13% (0.91%)
min_child_weight : 6.13% || max_depth : 2.01% || gamma : 4.62% || subsample : 0.84% || colsample_bytree : 0.65%
AUC : 85.34% (0.57%)
min_child_weight : 9.10% || max_depth : 6.62% || gamma : 7.85% || subsample : 0.68% || colsample_bytree : 0.45%
AUC : 86.58% (1.25%)
min_child_weight : 19.23% || max_depth : 6.43% || gamma : 0.00% || subsample : 0.80% || colsample_bytree : 0.34%
AUC : 89.35% (0.63%)
min_child_weight : 6.48% || max_depth : 2.45% || gamma : 2.84% || 

min_child_weight : 18.74% || max_depth : 9.30% || gamma : 0.96% || subsample : 0.87% || colsample_bytree : 0.91%
AUC : 88.95% (0.78%)
min_child_weight : 16.38% || max_depth : 9.90% || gamma : 2.78% || subsample : 0.71% || colsample_bytree : 0.93%
AUC : 87.63% (0.86%)
min_child_weight : 17.24% || max_depth : 7.46% || gamma : 2.94% || subsample : 0.75% || colsample_bytree : 0.75%
AUC : 87.71% (0.98%)
min_child_weight : 2.15% || max_depth : 7.32% || gamma : 9.69% || subsample : 0.55% || colsample_bytree : 0.55%
AUC : 85.45% (1.15%)
min_child_weight : 1.95% || max_depth : 3.81% || gamma : 6.55% || subsample : 0.56% || colsample_bytree : 0.81%
AUC : 85.42% (0.75%)
min_child_weight : 12.77% || max_depth : 8.00% || gamma : 6.07% || subsample : 0.98% || colsample_bytree : 0.18%
AUC : 87.45% (0.78%)
min_child_weight : 7.78% || max_depth : 2.25% || gamma : 6.75% || subsample : 0.77% || colsample_bytree : 0.64%
AUC : 84.98% (0.69%)
min_child_weight : 9.45% || max_depth : 8.89% || gamma : 7.41% ||

min_child_weight : 5.38% || max_depth : 6.14% || gamma : 6.93% || subsample : 0.65% || colsample_bytree : 0.21%
AUC : 86.81% (0.88%)
min_child_weight : 11.82% || max_depth : 9.78% || gamma : 4.18% || subsample : 0.51% || colsample_bytree : 0.97%
AUC : 86.97% (0.72%)
min_child_weight : 2.23% || max_depth : 9.29% || gamma : 4.17% || subsample : 0.83% || colsample_bytree : 0.47%
AUC : 88.14% (1.02%)
min_child_weight : 4.24% || max_depth : 4.86% || gamma : 1.46% || subsample : 0.62% || colsample_bytree : 0.39%
AUC : 88.01% (0.72%)
min_child_weight : 1.49% || max_depth : 3.53% || gamma : 5.84% || subsample : 0.89% || colsample_bytree : 0.89%
AUC : 86.15% (1.01%)
min_child_weight : 13.38% || max_depth : 8.49% || gamma : 7.75% || subsample : 0.58% || colsample_bytree : 0.87%
AUC : 86.37% (1.09%)
min_child_weight : 5.35% || max_depth : 9.38% || gamma : 0.26% || subsample : 0.80% || colsample_bytree : 0.84%
AUC : 89.48% (0.87%)
min_child_weight : 7.15% || max_depth : 4.30% || gamma : 0.11% || s

min_child_weight : 2.66% || max_depth : 4.54% || gamma : 0.96% || subsample : 0.51% || colsample_bytree : 0.83%
AUC : 88.29% (0.82%)
min_child_weight : 19.34% || max_depth : 7.86% || gamma : 2.24% || subsample : 0.64% || colsample_bytree : 0.26%
AUC : 88.44% (0.74%)
min_child_weight : 7.09% || max_depth : 6.54% || gamma : 8.47% || subsample : 0.64% || colsample_bytree : 0.89%
AUC : 85.58% (0.70%)
min_child_weight : 7.85% || max_depth : 5.19% || gamma : 5.41% || subsample : 0.69% || colsample_bytree : 0.96%
AUC : 86.47% (1.04%)
min_child_weight : 9.40% || max_depth : 4.32% || gamma : 7.62% || subsample : 0.64% || colsample_bytree : 0.16%
AUC : 86.61% (0.81%)
min_child_weight : 18.03% || max_depth : 7.14% || gamma : 0.45% || subsample : 0.57% || colsample_bytree : 0.38%
AUC : 89.63% (0.79%)
min_child_weight : 1.35% || max_depth : 7.03% || gamma : 5.81% || subsample : 0.54% || colsample_bytree : 0.50%
AUC : 86.59% (1.09%)
min_child_weight : 17.74% || max_depth : 9.46% || gamma : 0.29% || 

min_child_weight : 10.08% || max_depth : 2.02% || gamma : 0.06% || subsample : 0.78% || colsample_bytree : 0.90%
AUC : 87.00% (0.75%)
min_child_weight : 9.41% || max_depth : 9.81% || gamma : 0.49% || subsample : 0.52% || colsample_bytree : 0.28%
AUC : 89.39% (0.56%)
min_child_weight : 17.58% || max_depth : 7.77% || gamma : 2.36% || subsample : 0.82% || colsample_bytree : 0.46%
AUC : 88.55% (0.90%)
min_child_weight : 9.62% || max_depth : 6.71% || gamma : 0.93% || subsample : 0.80% || colsample_bytree : 0.97%
AUC : 88.83% (0.83%)
min_child_weight : 13.56% || max_depth : 9.67% || gamma : 0.60% || subsample : 0.77% || colsample_bytree : 0.75%
AUC : 89.15% (0.96%)
min_child_weight : 14.05% || max_depth : 8.06% || gamma : 0.71% || subsample : 0.94% || colsample_bytree : 0.56%
AUC : 89.46% (0.81%)
min_child_weight : 11.90% || max_depth : 9.08% || gamma : 0.22% || subsample : 0.79% || colsample_bytree : 0.68%
AUC : 89.68% (0.65%)
min_child_weight : 13.42% || max_depth : 9.17% || gamma : 0.64% 

...

min_child_weight : 13.65% || max_depth : 7.00% || gamma : 0.04% || subsample : 0.90% || colsample_bytree : 0.17%
AUC : 89.37% (0.75%)
min_child_weight : 10.99% || max_depth : 8.38% || gamma : 0.06% || subsample : 0.97% || colsample_bytree : 0.35%
AUC : 89.91% (0.64%)
min_child_weight : 6.79% || max_depth : 7.50% || gamma : 0.10% || subsample : 0.94% || colsample_bytree : 0.53%
AUC : 89.87% (0.58%)
min_child_weight : 7.35% || max_depth : 7.59% || gamma : 9.99% || subsample : 0.96% || colsample_bytree : 0.30%
AUC : 86.54% (1.17%)
min_child_weight : 15.03% || max_depth : 8.11% || gamma : 6.72% || subsample : 0.80% || colsample_bytree : 0.40%
AUC : 87.31% (0.81%)
min_child_weight : 9.59% || max_depth : 9.40% || gamma : 0.05% || subsample : 0.76% || colsample_bytree : 0.42%
AUC : 89.63% (0.57%)
min_child_weight : 9.29% || max_depth : 7.83% || gamma : 0.25% || subsample : 0.92% || colsample_bytree : 0.29%
AUC : 89.97% (0.43%)
min_child_weight : 8.90% || max_depth : 6.82% || gamma : 0.10% || 

min_child_weight : 11.87% || max_depth : 6.92% || gamma : 0.33% || subsample : 0.92% || colsample_bytree : 0.34%
AUC : 89.46% (0.46%)
min_child_weight : 5.38% || max_depth : 7.92% || gamma : 0.19% || subsample : 0.83% || colsample_bytree : 0.36%
AUC : 89.97% (0.61%)
min_child_weight : 2.58% || max_depth : 7.39% || gamma : 0.17% || subsample : 1.00% || colsample_bytree : 0.39%
AUC : 90.00% (0.71%)
min_child_weight : 3.92% || max_depth : 7.76% || gamma : 0.21% || subsample : 0.95% || colsample_bytree : 0.55%
AUC : 89.87% (0.94%)
min_child_weight : 9.35% || max_depth : 7.62% || gamma : 0.12% || subsample : 0.67% || colsample_bytree : 0.42%
AUC : 89.71% (0.73%)
min_child_weight : 16.00% || max_depth : 6.62% || gamma : 0.15% || subsample : 0.75% || colsample_bytree : 0.46%
AUC : 89.71% (0.85%)
min_child_weight : 2.72% || max_depth : 7.87% || gamma : 0.24% || subsample : 0.82% || colsample_bytree : 0.36%
AUC : 89.77% (0.79%)
min_child_weight : 7.27% || max_depth : 7.68% || gamma : 0.23% || s

min_child_weight : 9.02% || max_depth : 7.34% || gamma : 0.14% || subsample : 0.80% || colsample_bytree : 0.43%
AUC : 89.80% (0.56%)
min_child_weight : 8.73% || max_depth : 6.97% || gamma : 0.01% || subsample : 0.98% || colsample_bytree : 0.15%
AUC : 88.87% (0.47%)
min_child_weight : 13.52% || max_depth : 7.91% || gamma : 0.10% || subsample : 0.65% || colsample_bytree : 0.50%
AUC : 89.60% (0.84%)
min_child_weight : 3.13% || max_depth : 7.53% || gamma : 0.37% || subsample : 0.85% || colsample_bytree : 0.39%
AUC : 89.84% (0.72%)
min_child_weight : 5.52% || max_depth : 6.82% || gamma : 0.17% || subsample : 0.88% || colsample_bytree : 0.30%
AUC : 89.71% (0.44%)
min_child_weight : 6.81% || max_depth : 5.59% || gamma : 0.42% || subsample : 0.94% || colsample_bytree : 0.28%
AUC : 89.34% (0.67%)
min_child_weight : 2.70% || max_depth : 6.34% || gamma : 0.23% || subsample : 0.99% || colsample_bytree : 0.44%
AUC : 89.65% (0.70%)
min_child_weight : 2.30% || max_depth : 8.13% || gamma : 0.13% || su

min_child_weight : 10.41% || max_depth : 8.53% || gamma : 0.23% || subsample : 0.63% || colsample_bytree : 0.38%
AUC : 90.03% (0.82%)
min_child_weight : 4.07% || max_depth : 7.08% || gamma : 0.43% || subsample : 0.71% || colsample_bytree : 0.20%
AUC : 89.38% (0.46%)
min_child_weight : 9.81% || max_depth : 8.15% || gamma : 0.02% || subsample : 0.95% || colsample_bytree : 0.41%
AUC : 89.83% (0.59%)
min_child_weight : 2.10% || max_depth : 7.89% || gamma : 0.34% || subsample : 0.97% || colsample_bytree : 0.55%
AUC : 89.50% (0.68%)
min_child_weight : 2.39% || max_depth : 7.34% || gamma : 0.34% || subsample : 0.85% || colsample_bytree : 0.51%
AUC : 89.70% (0.89%)
min_child_weight : 6.57% || max_depth : 7.98% || gamma : 0.49% || subsample : 0.84% || colsample_bytree : 0.44%
AUC : 89.87% (0.67%)
min_child_weight : 5.67% || max_depth : 8.92% || gamma : 0.56% || subsample : 0.81% || colsample_bytree : 0.40%
AUC : 89.89% (0.66%)
min_child_weight : 7.96% || max_depth : 6.87% || gamma : 0.25% || su

(array([3.59439849, 7.48299792, 0.22649967, 0.93804315, 0.39789059]),
 9.634742823680398)

In [6]:
bounds = [(1,20), (2, 10), (0, 10), (0.5, 1), (0.1, 1)]

result = differential_evolution(XGB_Train_Model_using_KFold, bounds, maxiter = 5)
result.x, result.fun

min_child_weight : 8.86% || max_depth : 8.13% || gamma : 2.18% || subsample : 0.88% || colsample_bytree : 0.24%
AUC : 88.45% (0.59%)
min_child_weight : 2.27% || max_depth : 4.57% || gamma : 5.11% || subsample : 0.87% || colsample_bytree : 0.45%
AUC : 87.16% (0.94%)
min_child_weight : 11.60% || max_depth : 5.96% || gamma : 7.25% || subsample : 0.94% || colsample_bytree : 0.95%
AUC : 86.58% (1.03%)
min_child_weight : 16.37% || max_depth : 4.00% || gamma : 5.47% || subsample : 0.50% || colsample_bytree : 0.48%
AUC : 86.09% (0.90%)
min_child_weight : 4.86% || max_depth : 6.05% || gamma : 4.54% || subsample : 0.76% || colsample_bytree : 0.91%
AUC : 87.23% (0.96%)
min_child_weight : 16.63% || max_depth : 3.88% || gamma : 5.05% || subsample : 0.85% || colsample_bytree : 0.19%
AUC : 86.86% (0.68%)
min_child_weight : 13.64% || max_depth : 7.53% || gamma : 9.07% || subsample : 0.51% || colsample_bytree : 0.98%
AUC : 85.19% (0.89%)
min_child_weight : 1.04% || max_depth : 6.57% || gamma : 0.79% ||

min_child_weight : 19.74% || max_depth : 8.84% || gamma : 3.25% || subsample : 0.78% || colsample_bytree : 0.20%
AUC : 87.92% (0.95%)
min_child_weight : 18.75% || max_depth : 6.87% || gamma : 7.07% || subsample : 0.66% || colsample_bytree : 0.41%
AUC : 86.84% (1.01%)
min_child_weight : 14.11% || max_depth : 7.83% || gamma : 8.52% || subsample : 0.68% || colsample_bytree : 0.73%
AUC : 86.32% (1.17%)
min_child_weight : 7.18% || max_depth : 2.67% || gamma : 7.05% || subsample : 0.92% || colsample_bytree : 0.10%
AUC : 82.37% (2.33%)
min_child_weight : 5.20% || max_depth : 9.10% || gamma : 5.23% || subsample : 0.83% || colsample_bytree : 1.00%
AUC : 87.31% (0.83%)
min_child_weight : 17.41% || max_depth : 8.03% || gamma : 3.13% || subsample : 0.57% || colsample_bytree : 0.93%
AUC : 87.56% (1.00%)
min_child_weight : 11.04% || max_depth : 8.35% || gamma : 0.40% || subsample : 0.58% || colsample_bytree : 0.74%
AUC : 89.21% (0.78%)
min_child_weight : 6.86% || max_depth : 5.63% || gamma : 6.59% |

min_child_weight : 9.48% || max_depth : 7.98% || gamma : 1.83% || subsample : 0.62% || colsample_bytree : 0.72%
AUC : 88.17% (0.84%)
min_child_weight : 15.66% || max_depth : 9.07% || gamma : 2.34% || subsample : 0.81% || colsample_bytree : 0.25%
AUC : 88.48% (0.56%)
min_child_weight : 3.60% || max_depth : 9.23% || gamma : 0.80% || subsample : 0.99% || colsample_bytree : 0.79%
AUC : 89.10% (0.92%)
min_child_weight : 12.32% || max_depth : 3.83% || gamma : 3.59% || subsample : 0.99% || colsample_bytree : 0.43%
AUC : 87.12% (1.08%)
min_child_weight : 14.98% || max_depth : 7.81% || gamma : 8.94% || subsample : 0.65% || colsample_bytree : 0.59%
AUC : 86.14% (1.15%)
min_child_weight : 5.95% || max_depth : 3.32% || gamma : 8.77% || subsample : 0.80% || colsample_bytree : 0.49%
AUC : 85.46% (0.97%)
min_child_weight : 1.10% || max_depth : 7.04% || gamma : 0.55% || subsample : 0.91% || colsample_bytree : 0.77%
AUC : 89.41% (1.09%)
min_child_weight : 7.49% || max_depth : 8.57% || gamma : 6.74% || 

min_child_weight : 14.40% || max_depth : 3.05% || gamma : 5.31% || subsample : 0.82% || colsample_bytree : 0.20%
AUC : 86.79% (0.78%)
min_child_weight : 16.55% || max_depth : 3.95% || gamma : 1.17% || subsample : 0.83% || colsample_bytree : 0.11%
AUC : 84.95% (0.91%)
min_child_weight : 17.10% || max_depth : 6.55% || gamma : 4.45% || subsample : 0.89% || colsample_bytree : 0.60%
AUC : 87.77% (1.00%)
min_child_weight : 4.66% || max_depth : 3.28% || gamma : 6.75% || subsample : 0.90% || colsample_bytree : 0.52%
AUC : 86.24% (0.89%)
min_child_weight : 15.50% || max_depth : 6.72% || gamma : 1.65% || subsample : 0.72% || colsample_bytree : 0.70%
AUC : 88.24% (0.99%)
min_child_weight : 8.48% || max_depth : 8.38% || gamma : 1.29% || subsample : 0.73% || colsample_bytree : 0.97%
AUC : 88.36% (0.93%)
min_child_weight : 3.32% || max_depth : 2.90% || gamma : 2.73% || subsample : 0.84% || colsample_bytree : 0.36%
AUC : 86.17% (1.02%)
min_child_weight : 7.30% || max_depth : 4.46% || gamma : 2.43% ||

min_child_weight : 13.04% || max_depth : 8.73% || gamma : 9.45% || subsample : 0.70% || colsample_bytree : 0.41%
AUC : 86.08% (1.06%)
min_child_weight : 13.72% || max_depth : 9.50% || gamma : 3.02% || subsample : 0.80% || colsample_bytree : 0.34%
AUC : 88.32% (0.80%)
min_child_weight : 6.07% || max_depth : 2.15% || gamma : 0.67% || subsample : 0.87% || colsample_bytree : 0.19%
AUC : 86.53% (0.83%)
min_child_weight : 2.60% || max_depth : 9.55% || gamma : 0.67% || subsample : 0.86% || colsample_bytree : 0.31%
AUC : 89.49% (0.50%)
min_child_weight : 19.91% || max_depth : 7.90% || gamma : 0.95% || subsample : 0.97% || colsample_bytree : 0.43%
AUC : 89.16% (0.73%)
min_child_weight : 2.99% || max_depth : 8.77% || gamma : 1.08% || subsample : 0.82% || colsample_bytree : 0.41%
AUC : 89.44% (0.70%)
min_child_weight : 16.92% || max_depth : 9.01% || gamma : 1.03% || subsample : 0.79% || colsample_bytree : 0.18%
AUC : 88.96% (0.58%)
min_child_weight : 5.25% || max_depth : 7.55% || gamma : 0.10% ||

min_child_weight : 3.63% || max_depth : 9.22% || gamma : 4.22% || subsample : 0.66% || colsample_bytree : 0.20%
AUC : 87.79% (0.74%)
min_child_weight : 6.85% || max_depth : 7.50% || gamma : 1.11% || subsample : 0.70% || colsample_bytree : 0.31%
AUC : 89.03% (0.61%)
min_child_weight : 11.52% || max_depth : 9.55% || gamma : 2.54% || subsample : 0.98% || colsample_bytree : 0.41%
AUC : 88.60% (0.66%)
min_child_weight : 14.41% || max_depth : 8.20% || gamma : 1.91% || subsample : 0.99% || colsample_bytree : 0.37%
AUC : 88.73% (0.66%)
min_child_weight : 11.32% || max_depth : 2.65% || gamma : 9.58% || subsample : 0.90% || colsample_bytree : 0.81%
AUC : 84.58% (0.76%)
min_child_weight : 8.66% || max_depth : 8.60% || gamma : 5.87% || subsample : 0.61% || colsample_bytree : 0.30%
AUC : 86.72% (0.79%)
min_child_weight : 5.62% || max_depth : 8.79% || gamma : 8.74% || subsample : 0.79% || colsample_bytree : 0.47%
AUC : 86.53% (0.87%)
min_child_weight : 5.29% || max_depth : 8.32% || gamma : 1.78% || 

min_child_weight : 5.68% || max_depth : 9.06% || gamma : 4.64% || subsample : 0.92% || colsample_bytree : 0.27%
AUC : 87.90% (0.77%)
min_child_weight : 10.01% || max_depth : 9.72% || gamma : 1.12% || subsample : 0.70% || colsample_bytree : 0.92%
AUC : 88.71% (0.83%)
min_child_weight : 3.62% || max_depth : 8.75% || gamma : 0.06% || subsample : 0.80% || colsample_bytree : 0.23%
AUC : 89.58% (0.75%)
min_child_weight : 8.22% || max_depth : 9.54% || gamma : 5.16% || subsample : 0.88% || colsample_bytree : 0.16%
AUC : 87.55% (0.69%)
min_child_weight : 9.26% || max_depth : 7.96% || gamma : 1.64% || subsample : 0.95% || colsample_bytree : 0.71%
AUC : 88.62% (0.87%)
min_child_weight : 2.73% || max_depth : 7.36% || gamma : 1.02% || subsample : 0.74% || colsample_bytree : 0.43%
AUC : 89.17% (0.58%)
min_child_weight : 1.25% || max_depth : 7.43% || gamma : 7.43% || subsample : 0.85% || colsample_bytree : 0.99%
AUC : 86.43% (0.84%)
min_child_weight : 4.11% || max_depth : 8.35% || gamma : 2.55% || su

min_child_weight : 8.08% || max_depth : 8.87% || gamma : 0.28% || subsample : 0.74% || colsample_bytree : 0.22%
AUC : 89.70% (0.77%)
min_child_weight : 6.01% || max_depth : 7.81% || gamma : 1.80% || subsample : 0.71% || colsample_bytree : 0.75%
AUC : 88.24% (0.84%)
min_child_weight : 6.52% || max_depth : 8.61% || gamma : 3.64% || subsample : 0.65% || colsample_bytree : 0.43%
AUC : 87.86% (0.82%)
min_child_weight : 13.58% || max_depth : 8.00% || gamma : 5.99% || subsample : 0.77% || colsample_bytree : 0.78%
AUC : 87.21% (1.20%)
min_child_weight : 3.79% || max_depth : 8.25% || gamma : 0.23% || subsample : 0.79% || colsample_bytree : 0.40%
AUC : 89.73% (0.47%)
min_child_weight : 1.07% || max_depth : 4.21% || gamma : 6.33% || subsample : 0.90% || colsample_bytree : 0.24%
AUC : 87.23% (0.64%)
min_child_weight : 2.40% || max_depth : 7.80% || gamma : 4.06% || subsample : 0.97% || colsample_bytree : 0.47%
AUC : 88.37% (0.84%)
min_child_weight : 2.00% || max_depth : 2.99% || gamma : 1.31% || su

(array([1.34903689, 8.6324375 , 0.22517544, 0.8782863 , 0.34134826]),
 9.749625925952785)